# Part a

I0 = I1 + I5  

I2 = I3 + I4  

20 = 2I0 + 2I2 + 2I3  

2I1 = 2I3 + 2I5  

2I3 + I4 = 2I6  

I4 = 2I6 + I7  

I0 = I2 + I6  

I5 = I7

In [8]:
import numpy as np
import numpy.linalg as npl

In [21]:
# Define the coefficient matrix (A) and the constant vector (b)
# Part (a)
A = np.array([
    [1, -1, -1, 0, 0, 0, 0, 0],  # KCL 
    [0, 1, 0, 0, 0, 1, 0, -1],  # KCL
    [0, 0, 1, -1, -1, 0, 0, 0], # KCL 
    [0, 0, 0, 1, 0, -1, -1, 0], # KCL 
    [2, 0, 1, 0, 2, 0, 0, 0],   # KVL
    [0, -2, 1, 2, 0, 2, 0, 0],  # KVL 
    [0, 0, 0, 2, -1, 0, 2, 0],  # KVL 
    [0, 2, 0, 0, 0, 2, -2, 0]   # KVL
], dtype=float)
#checking singularity
det_A = np.linalg.det(A)
print(f"Determinant of matrix A: {det_A}")
b = np.array([0, 0, 0, 0, 20,0, 0, 0], dtype=float)


Determinant of matrix A: 340.0000000000001


# Part b
## Using Dense Matrix Technique
### LU Factorization

In [22]:
def forward_substitution(L, b):
    n = len(b)
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    return y

def backward_substitution(U, y):
    n = len(y)
    x = np.zeros_like(y)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

In [23]:

def LU_with_pivoting(A):
    n = len(A)    
    P = np.eye(n)  # Permutation matrix
    for k in range(n-1):
        max_row = np.argmax(abs(A[k:, k])) + k
        if A[max_row, k] == 0:
            raise ValueError("Matrix is singular!")
        A[[k, max_row]] = A[[max_row, k]]
        P[[k, max_row]] = P[[max_row, k]]
        for i in range(k+1, n):
            if A[k, k] != 0.0:
                mik = A[i, k] / A[k, k]
                A[i, k+1:n] = A[i, k+1:n] - mik * A[k, k+1:n]
                A[i, k] = mik
    return P, A

P, LU_matrix = LU_with_pivoting(A.copy())

L = np.tril(LU_matrix, k=-1) + np.eye(len(A))
U = np.triu(LU_matrix)

b_permuted = P @ b

y = forward_substitution(L, b_permuted)

x = backward_substitution(U, y)

print("Solution (currents):")
print(x)

Solution (currents):
[ 5.17647059  1.64705882  3.52941176  0.47058824  3.05882353 -0.58823529
  1.05882353  1.05882353]


# Part c
## Using Sparse Matrix Technique
### Gauss Seidal

In [24]:
# Gauss Seidel
from scipy import sparse
def sp_gauss_seidel(S, b, x, tol = 1.e-5, maxit = 100):
    n = len(b)
    err = 1.0
    iters = 0
    D = S.diagonal()
    xnew = np.zeros_like(x)
    while (err > tol and iters < maxit):
        iters += 1
        for i in range(n):
            rowstart = S.indptr[i]
            rowend = S.indptr[i+1]
            z = np.copy(x)
            z[:i] = xnew[:i]
            z[i]=0.0
            s = np.dot(S.data[rowstart:rowend], z[S.indices[rowstart:rowend]])
            xnew[i] = (b[i] - s) / D[i]
        err = np.linalg.norm(xnew-x,np.inf)
        x = np.copy(xnew)
    print('iterations required for convergence:', iters)
    return x


S = sparse.coo_matrix(A)
x = np.ones(8)
S = S.tocsr()
b = S.dot(x)
x = np.zeros(len(b))
x = sp_gauss_seidel(S, b, x, tol = 1.e-5, maxit = 100)
print(x)


iterations required for convergence: 3
[ inf -inf   9.  inf -inf -inf -inf  nan]


/tmp/ipykernel_77913/3013844619.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  xnew[i] = (b[i] - s) / D[i]


In [25]:
from scipy.sparse import csr_matrix
A_sparse = csr_matrix(A)

def gauss_seidel_sparse(A, b, x, tol=1.e-15, maxit=100):
    n = len(b)
    err = 1.0
    iters = 0
    xnew = np.zeros_like(x)
    while err > tol and iters < maxit:
        iters += 1
        for i in range(n):
            row_start = A.indptr[i]
            row_end = A.indptr[i + 1]
            row_indices = A.indices[row_start:row_end]
            row_data = A.data[row_start:row_end]

            sigma = sum(row_data[j] * xnew[row_indices[j]] for j in range(len(row_data)) if row_indices[j] != i)
            xnew[i] = (b[i] - sigma) / A[i, i]
        err = np.linalg.norm(xnew - x, np.inf)
        x = np.copy(xnew)
    print('Iterations required for convergence (sparse):', iters)
    return x

# Solve using Gauss-Seidel for sparse matrices
x0 = np.zeros_like(b)
x_gs_sparse = gauss_seidel_sparse(A_sparse, b, x0)
print("Gauss-Seidel (sparse) solution:", x_gs_sparse)

Iterations required for convergence (sparse): 3
Gauss-Seidel (sparse) solution: [ inf -inf   9.  inf -inf -inf -inf  nan]


/tmp/ipykernel_77913/3605600707.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  xnew[i] = (b[i] - sigma) / A[i, i]
/tmp/ipykernel_77913/3605600707.py:17: RuntimeWarning: invalid value encountered in scalar add
  sigma = sum(row_data[j] * xnew[row_indices[j]] for j in range(len(row_data)) if row_indices[j] != i)
